In [1]:
import pandas as pd
from openai import OpenAI
import openai
import os

In [2]:
dataset_path = 'Housing.csv'
client = OpenAI()

In [3]:
# Initialize the dataframe through the dataset.

df = pd.read_csv(dataset_path)

houses = []

# Convert the whole dataframe into array of dictionaries. Where each record is the dictionary. 

houses = df.to_dict(orient='records')

In [4]:
# Dividing records into categories that will be used further for prompt construction.

affordable_houses = [record for record in houses if 0 <= record.get("price", 0) <= 2000000]
moderate_houses = [record for record in houses if 2000000 <= record.get("price", 0) <= 4000000]
aboveAverage_houses = [record for record in houses if 4000000 <= record.get("price", 0) <= 6000000]
highEnd_houses = [record for record in houses if 6000000 <= record.get("price", 0) <= 8000000]
luxury_houses = [record for record in houses if 8000000 <= record.get("price", 0) <= 10000000]
premiumLuxury_houses = [record for record in houses if 10000000 <= record.get("price", 0) <= 12000000]

In [40]:
# houses_for_post = [] # Array of houses that will be used to generate posts.
# 
# # Applying filtering with specific feature.
# for record in affordable_houses:
#     if record['mainroad'] == 'yes' and record['area'] >= 3000 and record['bedrooms'] >= 3:
#         if (record['bedrooms'] >= 3):
#             filtered_record = {'affordable_houses': record}
#             houses_for_post.append(filtered_record)
# 
# # Applying filtering with specific feature.
# for record in moderate_houses:
#     if record['mainroad'] == 'yes' and record['area'] >= 3000 and record['bedrooms'] >= 3:
#         if record['basement'] == 'yes' and record['airconditioning'] == 'yes' and record['guestroom'] == 'yes':
#             filtered_record = {'moderate_houses': record}
#             houses_for_post.append(filtered_record)
# 
# # Applying filtering with specific feature.
# for record in aboveAverage_houses:
#     if record['mainroad'] == 'yes' and record['area'] >= 3000 and record['bedrooms'] >= 3:
#         if record['basement'] == 'yes' and record['airconditioning'] == 'yes' and record['guestroom'] == 'yes':
#             if record['furnishingstatus'] == 'furnished' and record['parking'] >= 1 and record['prefarea'] == 'yes':
#                 filtered_record = {'aboveAverage_houses': record}
#                 houses_for_post.append(filtered_record)
# 
# # Applying filtering with specific feature.
# for record in highEnd_houses:
#     if record['mainroad'] == 'yes' and record['area'] >= 3000 and record['bedrooms'] >= 3:
#         if record['basement'] == 'yes' and record['airconditioning'] == 'yes' and record['guestroom'] == 'yes':
#             if record['furnishingstatus'] == 'furnished' and record['parking'] >= 1 and record['prefarea'] == 'yes':
#                 filtered_record = {'highEnd_houses': record}
#                 houses_for_post.append(filtered_record)
# 
# # Applying filtering with specific feature.
# for record in luxury_houses:
#     if record['mainroad'] == 'yes' and record['area'] >= 3000 and record['bedrooms'] >= 3:
#         if record['basement'] == 'yes' and record['airconditioning'] == 'yes' and record['guestroom'] == 'yes':
#             if record['furnishingstatus'] == 'furnished' and record['parking'] >= 1 and record['prefarea'] == 'yes':
#                 filtered_record = {'luxury_houses': record}
#                 houses_for_post.append(filtered_record)
# 
# # Applying filtering with specific feature.
# for record in premiumLuxury_houses:
#     if record['mainroad'] == 'yes' and record['area'] >= 3000 and record['bedrooms'] >= 3:
#         if record['basement'] == 'yes' and record['airconditioning'] == 'yes' and record['guestroom'] == 'yes':
#             if record['furnishingstatus'] == 'furnished' and record['parking'] >= 1 and record['prefarea'] == 'yes':
#                 filtered_record = {'premiumLuxury_houses': record}
#                 houses_for_post.append(filtered_record)

# Instead of all this terrible code above, just replaced with one function

def filter_houses(record):
    return (
            record['mainroad'] == 'yes' and
            record['area'] >= 3000 and
            record['bedrooms'] >= 3 and
            record['basement'] == 'yes' and
            record['airconditioning'] == 'yes' and
            record['guestroom'] == 'yes' and
            record['furnishingstatus'] == 'furnished' or
            record['furnishingstatus'] == 'semifurnished' and
            record['parking'] >= 1 
    )

categories = ['affordable', 'moderate', 'aboveAverage', 'highEnd', 'luxury', 'premiumLuxury']

houses_for_post = []

for category in categories:
    category_houses = []
    for record in globals()[f"{category}_houses"]:
        if filter_houses(record):
            filtered_record = {f'{category}_houses': record}
            category_houses.append(filtered_record)
    houses_for_post.extend(category_houses)

In [27]:
# Again dividing the data about filtered houses by specific features to the variable data context.
# affordable_houses_data = houses_for_post[0].get('affordable_houses')
# moderate_houses_data = houses_for_post[0].get('moderate_houses')
# 
# aboveAverage_houses_data1 = houses_for_post[1].get('aboveAverage_houses')
# aboveAverage_houses_data2 = houses_for_post[2].get('aboveAverage_houses')
# aboveAverage_houses_data3 = houses_for_post[3].get('aboveAverage_houses')
# aboveAverage_houses_data4 = houses_for_post[4].get('aboveAverage_houses')
# 
# highEnd_houses_data = houses_for_post[5].get('highEnd_houses')
# 
# luxury_houses_data1 = houses_for_post[6].get('luxury_houses')
# luxury_houses_data2 = houses_for_post[7].get('luxury_houses')
# luxury_houses_data3 = houses_for_post[8].get('luxury_houses')
# luxury_houses_data4 = houses_for_post[9].get('luxury_houses')
# luxury_houses_data5 = houses_for_post[10].get('luxury_houses')
# premiumLuxury_houses_data = houses_for_post[11].get('luxury_houses')


                                    



#Prompts that used in generating posts.


# prompt_moderate_houses = f"Your mission is to make a positive and quote-provoking post, describing each data point about house. Moreover, you need enhance your post with unique approach in marketing or selling. Here are the data:\n{moderate_houses_data}\nGenerate a response:"
# prompt_aboveAverage_houses = f"Your mission is to make a positive and quote-provoking post, describing each data point about house. Moreover, you need enhance your post with unique approach in marketing or selling. Here are the data:\n{aboveAverage_houses_data}\nGenerate a response:"
# prompt_highEnd_houses = f"Your mission is to make a positive and quote-provoking post, describing each data point about house. Moreover, you need enhance your post with unique approach in marketing or selling. Here are the data:\n{highEnd_houses_data}\nGenerate a response:"
# prompt_luxury_houses1 = f"Your mission is to make a positive and quote-provoking post, describing each data point about house. Moreover, you need enhance your post with unique approach in marketing or selling. Here are the data:\n{luxury_houses_data1}\nGenerate a response:"
# prompt_luxury_houses2 = f"Your mission is to make a positive and quote-provoking post, describing each data point about house. Moreover, you need enhance your post with unique approach in marketing or selling. Here are the data:\n{luxury_houses_data2}\nGenerate a response:"

# This is how I previously initialized prompt data

In [46]:
# Generation of the post according to the role of context, and the main prompt from user. Below this section I created responses on each category, that initialized above, now I'm just iterating the whole list with data, and appending responses to the list. 

generated_posts = [] # list of generated posts

for record in houses_for_post:
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are the real estate agent who manage own instagram account for promoting houses to acquiring."},
            {"role": "user", "content": f"Your mission is to make a positive and quote-provoking post, describing each data point about house. Moreover, you need enhance your post with unique approach in marketing or selling. Here are the data:\n{record}\nGenerate a response:"}
        ]
    )
    generated_posts.append(completion.choices[0].message.content) # appending responses to list of generated posts

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-TRsdN***************************************lbPZ. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [ ]:
# # Generation of the post according to the role of context, and the main prompt from user.
# 
# 
# completion = client.chat.completions.create(
#   model="gpt-3.5-turbo",
#    messages=[
#     {"role": "system", "content": "You are the real estate agent who manage own instagram account for promoting houses to acquiring."},
#     {"role": "user", "content": prompt_moderate_houses}
#   ]
# )
# 
# generated_post_moderate_houses = completion.choices[0].message.content

In [ ]:
# Generation of the post according to the role of context, and the main prompt from user.

# completion = client.chat.completions.create(
#   model="gpt-3.5-turbo",
#    messages=[
#     {"role": "system", "content": "You are the real estate agent who manage own instagram account for promoting houses to acquiring."},
#     {"role": "user", "content": prompt_aboveAverage_houses}
#   ]
# )
# 
# generated_post_aboveAverage_houses = completion.choices[0].message.content

In [ ]:
# Generation of the post according to the role of context, and the main prompt from user.

# completion = client.chat.completions.create(
#   model="gpt-3.5-turbo",
#    messages=[
#     {"role": "system", "content": "You are the real estate agent who manage own instagram account for promoting houses to acquiring."},
#     {"role": "user", "content": prompt_highEnd_houses}
#   ]
# )
# 
# generated_post_highEnd_houses = completion.choices[0].message.content

In [ ]:
# Generation of the post according to the role of context, and the main prompt from user.

# completion = client.chat.completions.create(
#   model="gpt-3.5-turbo",
#    messages=[
#     {"role": "system", "content": "You are the real estate agent who manage own instagram account for promoting houses to acquiring."},
#     {"role": "user", "content": prompt_luxury_houses1}
#   ]
# )
# 
# generated_post_luxury_houses1 = completion.choices[0].message.content

In [ ]:
# Generation of the post according to the role of context, and the main prompt from user.

# completion = client.chat.completions.create(
#   model="gpt-3.5-turbo",
#    messages=[
#     {"role": "system", "content": "You are the real estate agent who manage own instagram account for promoting houses to acquiring."},
#     {"role": "user", "content": prompt_luxury_houses2}
#   ]
# )
# 
# generated_post_luxury_houses2 = completion.choices[0].message.content

In [ ]:
# This part of code, exists for generating md file with results

markdown_content = """# My Markdown File

This is some text in my Markdown file. You can include various formatting options such as:

- Lists
- *Italic text*
- **Bold text**
- [Links](https://www.example.com)
"""

file_path = "example.md"

# There is generation with divider - '-----'.

for post in generated_posts:
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(str(post) + "\n---\n")
    

print(f"Markdown file '{file_path}' has been generated.")

In [ ]:
# Converting md to txt. 

md_file_path = "example.md"
txt_file_path = "example.txt"

with open(md_file_path, "r", encoding="utf-8") as md_file:
    markdown_content = md_file.read()

with open(txt_file_path, "w", encoding="utf-8") as txt_file:
    txt_file.write(markdown_content)

print(f"Markdown file '{md_file_path}' has been converted to text file '{txt_file_path}'.")